<a href="https://colab.research.google.com/github/kamatsuoka/gpt-2/blob/master/gpt_2_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2 Playground

## Background
In this notebook you can play around with the medium (**345M** parameter) version of GPT-2.  of **Open AI's GPT-2** Model from the paper **[Language Models are Unsupervised Multitask Learners](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf)**

## Steps
Before starting, is recommended to set *Runtime Type* to *GPU* on the top menu bar.

###1. Installation
Download the model data and install Python libraries. 


In [0]:
import os
basedir = os.getcwd()

In [0]:
os.chdir(basedir)
!rm -rf gpt-2
!git clone -q https://github.com/kamatsuoka/gpt-2/
os.chdir('gpt-2')

In [0]:
!pip3 install --quiet --user pipenv

In [18]:
import site
os.environ['PATH'] += os.pathsep + site.USER_BASE + "/bin" # to find pipenv
!pipenv --bare install


Creating a virtualenv for this project…
Pipfile: /content/gpt-2/Pipfile
Using /usr/local/bin/python (3.6.7) to create virtualenv…
⠙ Creating virtual environment...Using base prefix '/usr'
New python executable in /root/.local/share/virtualenvs/gpt-2-cJGb30tS/bin/python3
Also creating executable in /root/.local/share/virtualenvs/gpt-2-cJGb30tS/bin/python
Installing setuptools, pip, wheel...
done.
Running virtualenv with interpreter /usr/local/bin/python

✔ Successfully created virtual environment! 
Virtualenv location: /root/.local/share/virtualenvs/gpt-2-cJGb30tS
Pipfile.lock (706bb4) out of date, updating to (061490)…
Locking [dev-packages] dependencies…
Locking [packages] dependencies…
✔ Success! 
Updated Pipfile.lock (706bb4)!
Installing dependencies from Pipfile.lock (706bb4)…
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 26/26 — 00:00:23
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [17]:
!pip3 --quiet install pipenv
!pipenv install
!pipenv shell
!python download_model.py 345M --quiet

/bin/bash: pipenv: command not found
/bin/bash: pipenv: command not found
Traceback (most recent call last):



## Conditional sample generation

To generate conditional samples from the small model:
```
!python3 src/interactive_conditional_samples.py
```
It comes with a few flags available, with a default value: 
- `seed = None`  || a random value is generated unless specified. give a specific integer value if you want to reproduce same results in the future.
- `nsamples = 1`     ||  specify the number of samples you want to print
- `length = None`   ||  number of tokens (words) to print on each sample.
- `batch_size= 1`  ||  how many inputs you want to process simultaneously. *only affects speed/memory* 
- `temperature = 1`  ||  float between 0 and 1. scales logits before sampling prior to softmax. higher temperature results in more random completions.
- `top_k = 0`   ||  Integer value controlling diversity.  Truncates the set of logits considered to those with the highest values. 1 means only 1 word is considered for each step (token), resulting in deterministic completions. 40 means 40 words are considered at each step. 0 (default) is a special setting meaning no restrictions. 40 generally is a good value.


In [0]:
!python3 src/interactive_conditional_samples.py --model_name='345M' --nsamples=2 --top_k=40 --temperature=.80

In [0]:
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=2 --top_k=100 --temperature=1

2019-05-17 01:10:52.485283: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-17 01:10:52.485488: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2118680 executing computations on platform Host. Devices:
2019-05-17 01:10:52.485519: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-17 01:10:52.643805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-17 01:10:52.644315: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2117fa0 executing computations on platform CUDA. Devices:
2019-05-17 01:10:52.644346: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-17 01:10:52.644717: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

### 2. Question-Answering

- Context: passage, some question/answer pairs, and token `A:`
- For a single word answer (i.e.: Yes/No, city), set flag `length=1`

Sample prompt 1 ([*The Baseline test*](https://bladerunner.fandom.com/wiki/Baseline_Test))
```
Q: What's it like to hold the hand of someone you love? 
A: Interlinked. 
Q: Do they teach you how to feel finger to finger? 
A: Interlinked. 
Q: Do you long for having your heart interlinked? 
A: 
```

Sample prompt 2: 
```
The 2008 Summer Olympics torch relay was run from March 24 until August 8, 2008, prior to the 2008 Summer
Olympics, with the theme of “one world, one dream”. Plans for the relay were announced on April 26, 2007, in
Beijing, China. The relay, also called by the organizers as the “Journey of Harmony”, lasted 129 days and carried
the torch 137,000 km (85,000 mi) – the longest distance of any Olympic torch relay since the tradition was started
ahead of the 1936 Summer Olympics.
After being lit at the birthplace of the Olympic Games in Olympia, Greece on March 24, the torch traveled to the Panathinaiko Stadium in Athens, and then to Beijing, arriving on March 31. From Beijing, the torch was
following a route passing through six continents. The torch has visited cities along the Silk Road, symbolizing
ancient links between China and the rest of the world. The relay also included an ascent with the flame to the top of
Mount Everest on the border of Nepal and Tibet, China from the Chinese side, which was closed specially for the
event.
Q: What was the length of the race?
A: 137,000 km
Q: Was it larger than previous ones?
A: No
Q: Where did the race begin?
A: Olympia, Greece
Q: Where did they go after?
A: Athens
Q: How many days was the race?
A: seven
Q: Did they visit any notable landmarks?
A: Panathinaiko Stadium
Q: And did they climb any mountains?
A:
```



In [0]:
!python3 src/interactive_conditional_samples.py  --model_name='345M'  --nsamples=10 --top_k=40 --temperature=.80 --length=1

2019-05-17 01:13:26.642567: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-17 01:13:26.642778: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1de2680 executing computations on platform Host. Devices:
2019-05-17 01:13:26.642808: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-17 01:13:26.799442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-17 01:13:26.799946: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1de1fa0 executing computations on platform CUDA. Devices:
2019-05-17 01:13:26.799976: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-17 01:13:26.800303: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

### 3. Summarization



- Context: article and text *`TL;DR:`* or *`Summary:`* at the end.

Sample prompt:

```
Theodore McCarrick is the most senior Catholic figure to be dismissed from the priesthood in modern times.
US Church officials said allegations he had sexually assaulted a teenager five decades ago were credible.
Mr McCarrick, 88, had previously resigned but said he had "no recollection" of the alleged abuse.
"No bishop, no matter how influential, is above the law of the Church," Cardinal Daniel DiNardo, president of the United States Conference of Catholic Bishops said in a statement.
"For all those McCarrick abused, I pray this judgment will be one small step, among many, toward healing."
The alleged abuses may have taken place too long ago for criminal charges to be filed because of the statute of limitations.
Mr McCarrick was the archbishop of Washington DC from 2001 to 2006. Since his resignation last year from the College of Cardinals, he has been living in seclusion in a monastery in Kansas.
He was the first person to resign as a cardinal since 1927.
He is among hundreds of members of the clergy accused of sexually abusing children over several decades and his dismissal comes days before the Vatican hosts a summit on preventing child abuse.
The Vatican said Pope Francis had ruled Mr McCarrick's expulsion from the clergy as definitive, and would not allow any further appeals against the decision. 
TL;DR: 
```



In [0]:
!python3 src/interactive_conditional_samples.py --nsamples=3 --length=100 --temperature=1 

2019-02-17 01:22:47.097283: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-02-17 01:22:47.097552: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x28e2680 executing computations on platform Host. Devices:
2019-02-17 01:22:47.097596: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-02-17 01:22:47.185157: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-02-17 01:22:47.185826: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x28e35a0 executing computations on platform CUDA. Devices:
2019-02-17 01:22:47.185870: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla K80, Compute Capability 3.7
2019-02-17 01:22:47.186287: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found d

### 4. Translation



- Context: a few example pairs of the format *`english_sentence = spanish_sentence`*, and then *`english_sentence =`*  at the end. 

Sample prompt:
```
Good morning. = Buenos días.
I am lost. Where is the restroom? = Estoy perdido. ¿Dónde está el baño?
How much does it cost? = ¿Cuánto cuesta?
How do you say maybe in Spanish? = ¿Cómo se dice maybe en Español?
Would you speak slower, please. = Por favor, habla mas despacio.
Where is the book store? = ¿Dónde está la librería?
At last a feminist comedian who makes jokes about men. = Por fin un cómico feminista que hace chistes sobre hombres.

How old are you? = 


```


In [0]:
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=3 --temperature=1

2019-05-17 01:15:18.550026: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-17 01:15:18.550239: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x3476680 executing computations on platform Host. Devices:
2019-05-17 01:15:18.550270: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-17 01:15:18.708562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-17 01:15:18.709079: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x3475fa0 executing computations on platform CUDA. Devices:
2019-05-17 01:15:18.709112: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-17 01:15:18.709483: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de